# Make an SED!

In [1]:
from SEDkit import sed, spectrum, synphot
import astropy.units as q

Let's create a new SED for an M-dwarf .

In [2]:
s = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')

Make this handle asymmetric uncertainties!


Now let's add some photometry! And then take a look at how it's stored.

In [3]:
s.add_photometry('2MASS.J', 14.391, 0.029)
s.add_photometry('2MASS.H', 13.677, 0.032)
s.add_photometry('2MASS.Ks', 13.535, 0.034)
s.add_photometry('WISE.W1', 13.55, 0.027)
s.add_photometry('WISE.W2', 13.581, 0.035)
s.add_photometry('WISE.W3', 12.884, None)
s.add_photometry('SDSS.u', 20.486, 0.073)
s.add_photometry('SDSS.g', 17.779, 0.006)
s.add_photometry('SDSS.r', 16.431, 0.004)
s.add_photometry('SDSS.i', 15.843, 0.004)
s.add_photometry('SDSS.z', 15.514, 0.005)

In [4]:
s.photometry

band,eff,app_magnitude,app_magnitude_unc,app_flux,app_flux_unc,abs_magnitude,abs_magnitude_unc,abs_flux,abs_flux_unc,bandpass
,um,,,erg / (Angstrom cm2 s),erg / (Angstrom cm2 s),,,erg / (Angstrom cm2 s),erg / (Angstrom cm2 s),
str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,object
SDSS.u,3556.523841771216,20.486,0.073,2.325646727179245e-17,1.5636598497874093e-18,67092.551,3847.73,0.0,0.0,SDSS.u
SDSS.g,4702.495238900329,17.779,0.006,4.270284435615815e-16,2.3598463882304325e-18,58227.007,3339.293,0.0,0.0,SDSS.g
SDSS.r,6175.578617515661,16.431,0.004,6.769406339759335e-16,2.4939414602158772e-18,53812.248,3086.108,0.0,0.0,SDSS.r
SDSS.i,7489.976493830274,15.843,0.004,6.483971058276448e-16,2.3887832163507487e-18,51886.522,2975.669,0.0,0.0,SDSS.i
SDSS.z,8946.70937532557,15.514,0.005,5.295126280938415e-16,2.438495768001959e-18,50809.032,2913.875,0.0,0.0,SDSS.z
2MASS.J,12393.092661933399,14.391,0.029,5.490034894182187e-16,1.4663876108538818e-17,47131.158,2702.951,0.0,0.0,2MASS.J
2MASS.H,16494.94684561825,13.677,0.032,3.758881379010941e-16,1.1078584613943674e-17,44792.777,2568.846,0.0,0.0,2MASS.H
2MASS.Ks,21638.605296287602,13.535,0.034,1.65314523412906e-16,5.176850298817227e-18,44327.72,2542.176,0.0,0.0,2MASS.Ks


In [5]:
s.results

param,value,unc,units
object,object,object,object
name,2MASS 261832886,--,--
age,--,--,--
distance,572.28,16.41,pc
parallax,1.7474,0.0501,mas
radius,--,--,--
spectral_type,60.0,0.5,--
membership,--,--,--
fbol,9.41e-08,8.88e-10,erg / (cm2 s)
mbol,6.085,0.01,--


But there's no radius so let's estimate from the Lbol and age...